In [19]:
from import_src import import_src
import_src()

Append /home/v_user/khiempm/MLOpsVN/src to sys.path


In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
import mlflow
import seaborn as sns
import time
import pickle
from utils.utils import save_request_data, handle_prediction
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

In [21]:
pd.set_option('display.max_columns', 500)

In [22]:
link_data_test_raw_32 = '../../data/captured_data/phase-3/prob-2/raw/'
link_data_test_raw_22 = '../../data/captured_data/phase-2/prob-2/raw/'
link_data_test_processed_32 = '../../data/captured_data/phase-3/prob-1/'
train_raw_data_32 = '../../data/raw_data/phase-3/prob-2/raw_train.parquet'
train_raw_data_22 = '../../data/raw_data/phase-2/prob-2/raw_train.parquet'
link_y_data_32 = '../../data/train_data/phase-3/prob-2/train_y.parquet'
link_train_data_32 = '../../data/train_data/phase-3/prob-2/train_x.parquet'
link_test_data_32 = '../../data/train_data/phase-3/prob-2/test_x.parquet'
link_y_test_data_32 =   '../../data/train_data/phase-3/prob-2/test_y.parquet'
link_y_data_22 = '../../data/train_data/phase-2/prob-2/train_y.parquet'
link_train_data_22 = '../../data/train_data/phase-2/prob-2/train_x.parquet'


In [23]:
def combine_parquet_test(url_test, parquet=False):
    raw_test = pd.DataFrame()
    count_ = 0
    for file_path in os.listdir(url_test):
        if 'parquet' not in file_path or "123.parquet" in file_path:
            continue
        captured_data = pd.read_parquet(f'{url_test}{file_path}')
        if parquet:
            captured_data['parquet_idx'] = count_
        raw_test = pd.concat([raw_test, captured_data])
        count_ += 1
    print(count_)
    return raw_test.reset_index(drop=True)

In [25]:
raw_test_32 = combine_parquet_test(link_data_test_raw_32)
processed_test_32 = combine_parquet_test(link_data_test_processed_32)

200
200


In [26]:
raw_train_22 = pd.read_parquet(train_raw_data_22)
raw_train_32 = pd.read_parquet(train_raw_data_32)
order_cols = raw_train_32.columns.tolist()
order_cols.remove('label')
raw_test_32 = raw_test_32[order_cols].reset_index(drop=True)
processed_test_32 = processed_test_32[order_cols].reset_index(drop=True)
 
# captured_data = captured_data[order_cols].reset_index(drop=True)

In [23]:
raw_train_22.to_parquet("../../data/external_data/phase-3/prob-2/external_data.parquet", index=False)

In [37]:
os.environ.get("MLFLOW_TRACKING_URI")

'http://localhost:5000'

In [8]:
# mlflow_evn = os.environ.get("MLFLOW_TRACKING_URI")
mlflow_evn = "http://localhost:5000"
mlflow.set_tracking_uri(mlflow_evn)

In [13]:
model_uri = os.path.join(
    "models:/", "phase-3_prob-2_lgbm_cv_specific_handle", "10"
)
input_schema = mlflow.models.Model.load(model_uri).get_input_schema().to_dict()
model = mlflow.sklearn.load_model(model_uri)
get_features = [each['name'] for each in input_schema]

In [16]:
model_uri = os.path.join(
    "models:/", "phase-3_prob-1_lgbm_cv_specific_handle", "4"
)
model_binary = mlflow.sklearn.load_model(model_uri)

In [10]:
# x_valid_set = pd.read_parquet(link_test_data_32)
# y_valid_set = pd.read_parquet(link_y_test_data_32)
x_valid_set = pd.read_parquet(link_train_data_32)
y_valid_set = pd.read_parquet(link_y_data_32)
feature_columns = x_valid_set.columns.tolist()

In [17]:
processed_test_32['pred_label'] = model.predict(processed_test_32[get_features])
processed_test_32['binary_label'] = model_binary.predict(processed_test_32[get_features])

In [18]:
processed_test_32['pred_label'].value_counts() / processed_test_32.shape[0]

pred_label
Normal                   0.368356
Other                    0.230117
Exploits                 0.208095
Denial of Service        0.140141
Information Gathering    0.045332
Malware                  0.007960
Name: count, dtype: float64

In [19]:
processed_test_32['binary_label'].value_counts() / processed_test_32.shape[0]

binary_label
1    0.63637
0    0.36363
Name: count, dtype: float64

In [15]:
# combine_data = x_valid_set.copy()
# combine_data['label'] = y_valid_set
# combine_data = data_processing_phase3(combine_data, 'label')
# combine_data[(combine_data['feature2'].isna()) & (combine_data['feature3'].isna()) & (combine_data['feature4'].isna())].shape

In [33]:
raw_train_32[raw_train_32['label'].isin(['Denial of Service'])]['feature7'].value_counts()

feature7
200.0       4574
104.0        725
534.0        452
1064.0       344
756.0        319
            ... 
140627.0       1
3114.0         1
48858.0        1
2090.0         1
22916.0        1
Name: count, Length: 1716, dtype: int64

In [34]:
raw_train_32[raw_train_32['label'].isin(['Exploits'])]['feature7'].value_counts()

feature7
200.0      4596
364.0       663
180.0       215
608.0       132
1280.0       75
           ... 
22071.0       1
2922.0        1
41437.0       1
18174.0       1
41218.0       1
Name: count, Length: 3266, dtype: int64

In [31]:
raw_train_32[raw_train_32['label'].isin(['Normal'])]['feature4'].value_counts()

feature4
FIN    21955
CON     6794
INT     3746
REQ      960
RST       30
ECO        3
ACC        1
no         1
Name: count, dtype: int64

In [22]:
a = x_valid_set.groupby(feature_columns).agg(unique_feauture = ("label", 'nunique')).reset_index()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,unique_feauture
0,0.000000,6,0,1,2.0,2.0,56.0,92.0,0.0,0.0,5.626914,9.244215,0.0,0.0,39808.512,39808.492,0.000000,8.922835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,46.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,1
1,0.000000,6,0,1,2.0,2.0,56.0,92.0,0.0,0.0,6.157446,10.115804,0.0,0.0,36378.560,36378.588,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,46.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,1
2,0.000000,6,0,1,3.0,3.0,84.0,138.0,0.0,0.0,8.704350,14.300004,0.0,0.0,25734.168,25734.172,2100.499000,2100.492250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,46.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,1
3,0.000000,6,0,4,1.0,0.0,46.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1
4,0.000000,6,0,4,1.0,0.0,46.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50649,59.999947,6,0,4,2.0,0.0,92.0,0.0,0.0,0.0,6.133339,0.000000,0.0,0.0,60000.648,0.000,17.334842,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1
50650,59.999966,6,0,4,2.0,0.0,92.0,0.0,0.0,0.0,6.133337,0.000000,0.0,0.0,60000.700,0.000,10.979212,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1
50651,59.999985,6,0,4,2.0,0.0,92.0,0.0,0.0,0.0,6.133335,0.000000,0.0,0.0,60000.648,0.000,0.662000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1
50652,59.999989,6,0,4,2.0,0.0,92.0,0.0,0.0,0.0,6.133335,0.000000,0.0,0.0,60000.640,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1


In [100]:
def get_predict_data(data, model):
    for each in ['label', 'pred', 'pred_1', 'proba', 'proba_1', 'proba_2']:
        try:
            data = data.drop(columns=[each])
        except:
            pass
    class_ = model.classes_
    pred_proba = model.predict_proba(data.values)
    data['pred'] = model.predict(data.values)
    partitioned_indices = np.argpartition(pred_proba, -2)[:, -2]
    data['pred_1'] = [class_[idx] for idx in partitioned_indices]
    data['proba'] = np.max(pred_proba, axis=1)
    data['proba_1'] = np.sort(pred_proba, axis=1)[:, -2]
    data['proba_2'] = np.sort(pred_proba, axis=1)[:, -3]
    return data

In [101]:
x_valid_set = get_predict_data(x_valid_set, model)
x_valid_set['label'] = y_valid_set

In [102]:
x_valid_set['label'].value_counts()

label
Normal                   41803
Exploits                 13507
Denial of Service        11594
Other                     5132
Information Gathering     4444
Malware                    932
Name: count, dtype: int64

In [103]:
x_valid_set[x_valid_set['label'] == x_valid_set['pred']].shape[0] / x_valid_set.shape[0]

0.905570195835271

In [104]:
x_valid_set[(x_valid_set['pred'] == 'Normal') & (x_valid_set['proba'] < 0.5) & (x_valid_set['pred'] == x_valid_set['label'])]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,pred,pred_1,proba,proba_1,proba_2,label
290,0.716627,112,0,3,10.0,6.0,2062.0,268.0,254.0,252.0,2.071929e+04,2500.603516,2.0,1.0,75.265222,131.351594,3867.576182,200.581156,255.0,6.597428e+07,3.598944e+08,255.0,0.165191,0.059867,0.105324,206.0,45.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,Normal,Denial of Service,0.470930,0.467584,0.058353,Normal
867,0.000008,42,0,4,2.0,0.0,180.0,0.0,254.0,0.0,9.000000e+07,0.000000,0.0,0.0,0.008000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,90.0,0.0,0.0,0.0,4.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,Normal,Malware,0.377805,0.375765,0.107929,Normal
1322,0.600038,112,5,3,10.0,8.0,840.0,1100.0,62.0,252.0,1.007936e+04,12839.187500,2.0,2.0,66.670889,75.857859,3538.246626,108.831766,255.0,2.491970e+09,9.864645e+08,255.0,0.132973,0.068218,0.064755,84.0,138.0,1.0,129.0,2.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,Normal,Exploits,0.391955,0.317359,0.220877,Normal
1387,0.471622,112,0,3,10.0,8.0,488.0,354.0,254.0,252.0,7.463604e+03,5258.448730,2.0,1.0,49.981667,59.472145,3522.199061,108.134969,255.0,3.996589e+09,9.110970e+08,255.0,0.106895,0.055307,0.051588,49.0,44.0,0.0,0.0,7.0,1.0,2.0,2.0,1.0,6.0,0.0,0.0,0.0,3.0,6.0,0.0,Normal,Denial of Service,0.498385,0.492119,0.006892,Normal
1543,0.000010,118,0,4,2.0,0.0,1416.0,0.0,254.0,0.0,5.664000e+08,0.000000,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,708.0,0.0,0.0,0.0,4.0,2.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,2.0,4.0,0.0,Normal,Denial of Service,0.488514,0.476085,0.030815,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76308,0.986863,112,5,3,10.0,10.0,818.0,1256.0,62.0,252.0,5.974487e+03,9168.445312,2.0,2.0,109.651444,104.886219,7507.617394,168.517266,255.0,4.246165e+09,1.464327e+09,255.0,0.159748,0.042221,0.117527,82.0,126.0,1.0,182.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,Normal,Exploits,0.458837,0.322624,0.153452,Normal
76380,0.281343,112,5,3,10.0,6.0,852.0,268.0,254.0,252.0,2.180968e+04,6369.449219,2.0,1.0,31.260333,52.869602,1484.573977,74.091555,255.0,6.157439e+08,2.497807e+09,255.0,0.066560,0.016240,0.050320,85.0,45.0,1.0,0.0,5.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,1.0,1.0,4.0,0.0,Normal,Information Gathering,0.466949,0.359487,0.152455,Normal
76536,0.000008,118,0,4,2.0,0.0,104.0,0.0,254.0,0.0,5.200000e+07,0.000000,0.0,0.0,0.008000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,52.0,0.0,0.0,0.0,4.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,3.0,2.0,0.0,Normal,Denial of Service,0.499618,0.493258,0.004691,Normal
76787,0.749725,112,5,3,60.0,14.0,68199.0,612.0,254.0,252.0,7.156011e+05,6071.559570,27.0,1.0,12.707203,54.558691,1324.540809,72.099187,255.0,3.433636e+09,1.775659e+09,255.0,0.078087,0.039859,0.038228,1137.0,44.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,Normal,Denial of Service,0.488007,0.464458,0.046092,Normal


In [19]:
x_valid_set[x_valid_set['label'] != x_valid_set['pred']]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,pred,proba,proba_1,label
16,1.295045,112,0,3,38.0,10.0,41294.0,440.0,254.0,252.0,2.483806e+05,2446.247070,16.0,1.0,34.976756,130.816664,3526.286748,239.805703,255.0,2.903390e+09,6.752542e+08,255.0,0.259867,0.117686,0.142181,1087.0,44.0,0.0,0.0,3.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,Normal,0.572259,0.381738,Denial of Service
25,0.869466,112,5,3,10.0,10.0,792.0,5860.0,62.0,252.0,6.560349e+03,48526.335940,2.0,3.0,96.607333,91.079000,5681.958697,7781.593811,255.0,4.138992e+09,1.410473e+09,255.0,0.138996,0.040246,0.098750,79.0,586.0,1.0,2486.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,Exploits,0.949532,0.047229,Denial of Service
27,0.000023,118,0,4,2.0,0.0,1064.0,0.0,254.0,0.0,1.850435e+08,0.000000,0.0,0.0,0.023000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,532.0,0.0,0.0,0.0,34.0,2.0,14.0,14.0,1.0,34.0,0.0,0.0,0.0,14.0,34.0,0.0,Normal,0.843798,0.154542,Denial of Service
28,1.270563,112,3,3,12.0,12.0,632.0,682.0,254.0,252.0,3.651924e+03,3941.559814,3.0,4.0,110.171724,107.731453,5941.325871,149.239000,255.0,2.773647e+09,2.821260e+09,255.0,0.196496,0.085516,0.110980,53.0,57.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,Normal,0.631007,0.359905,Denial of Service
30,0.772133,112,9,3,30.0,20.0,19574.0,1694.0,254.0,252.0,1.960491e+05,16681.064450,12.0,7.0,26.625276,40.295262,1753.288116,51.647203,255.0,3.654671e+09,4.294325e+09,255.0,0.052295,0.005933,0.046362,652.0,85.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,Denial of Service,0.599857,0.289460,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77325,0.570350,112,5,3,10.0,6.0,804.0,268.0,254.0,252.0,1.015517e+04,3141.930420,2.0,1.0,60.862333,97.026203,3193.094357,154.250516,255.0,2.666389e+08,1.609670e+08,255.0,0.198407,0.085216,0.113191,80.0,45.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,3.0,1.0,0.0,Information Gathering,0.541173,0.406464,Exploits
77339,0.449695,112,0,3,10.0,6.0,1014.0,268.0,254.0,252.0,1.624212e+04,3984.923096,2.0,1.0,49.874889,70.835203,2633.616420,117.241398,255.0,3.284473e+09,3.122467e+09,255.0,0.174442,0.095511,0.078931,101.0,45.0,0.0,0.0,7.0,1.0,2.0,3.0,1.0,4.0,0.0,0.0,0.0,3.0,4.0,0.0,Normal,0.501162,0.479697,Denial of Service
77386,0.806158,112,5,3,14.0,8.0,6806.0,354.0,254.0,252.0,6.271723e+04,3076.320068,4.0,1.0,57.923156,107.768859,4157.446076,175.440797,255.0,6.993242e+08,3.574417e+09,255.0,0.167647,0.051767,0.115880,486.0,44.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,1.0,4.0,2.0,0.0,Exploits,0.625951,0.302411,Denial of Service
77388,0.000003,118,0,4,2.0,0.0,104.0,0.0,254.0,0.0,1.386667e+08,0.000000,0.0,0.0,0.003000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,52.0,0.0,0.0,0.0,10.0,2.0,5.0,5.0,1.0,10.0,0.0,0.0,0.0,5.0,10.0,0.0,Denial of Service,0.511008,0.486724,Normal


In [ ]:
x_valid_set['pred'] = handle_prediction(x_valid_set['pred'].values, x_valid_set['proba'].values)

In [38]:
x_valid_set[(x_valid_set['label'] != x_valid_set['pred']) & (x_valid_set['pred'] == 'Normal')]['label'].value_counts()

label
Denial of Service        661
Exploits                  73
Information Gathering     23
Name: count, dtype: int64

In [15]:
processed_test_32[(processed_test_32["feature2"] == -1) & (processed_test_32["feature3"] == -1)
                  & (processed_test_32["feature4"] == -1)]["label"].value_counts()

label
Exploits                 1701
Normal                   1251
Other                    1071
Denial of Service         666
Information Gathering     216
Name: count, dtype: int64

In [17]:
processed_test_32

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label
0,1.169722,112,9,3,52.0,42.0,37210.0,3380.0,31.0,29.0,2.495978e+05,2.256947e+04,18.0,8.0,22.928980,28.516902,1909.633851,61.133949,255.0,3.463648e+09,3.532505e+09,255.0,0.000674,0.000522,0.000152,716.0,80.0,0.0,0.0,1.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,Normal
1,0.000004,77,0,4,2.0,0.0,180.0,0.0,254.0,0.0,1.800000e+08,0.000000e+00,0.0,0.0,0.004000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,90.0,0.0,0.0,0.0,7.0,2.0,7.0,7.0,7.0,7.0,0.0,0.0,0.0,7.0,7.0,0.0,Exploits
2,0.001025,118,2,1,2.0,2.0,130.0,162.0,31.0,29.0,5.073171e+05,6.321951e+05,0.0,0.0,0.009000,0.007000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,65.0,81.0,0.0,0.0,2.0,0.0,4.0,1.0,1.0,4.0,0.0,0.0,0.0,2.0,3.0,0.0,Normal
3,2.475373,112,9,3,42.0,22.0,31283.0,1826.0,62.0,252.0,9.869705e+04,5.633090e+03,18.0,8.0,60.374949,112.654289,4537.055399,151.972969,255.0,2.981134e+09,5.506478e+08,255.0,0.145429,0.074754,0.070675,745.0,83.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,Exploits
4,58.941711,118,2,5,86.0,0.0,6708.0,0.0,254.0,0.0,8.998721e+02,0.000000e+00,0.0,0.0,695.143563,0.000000,795.381562,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,78.0,0.0,0.0,0.0,2.0,6.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,Denial of Service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109367,0.000005,118,2,-1,2.0,0.0,114.0,0.0,254.0,0.0,9.120000e+07,0.000000e+00,0.0,0.0,0.005000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,17.0,2.0,16.0,16.0,16.0,17.0,0.0,0.0,0.0,16.0,17.0,0.0,Other
109368,0.000010,-1,-1,-1,2.0,0.0,168.0,0.0,254.0,0.0,6.720000e+07,0.000000e+00,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,84.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,19.0,1.0,0.0,Information Gathering
109369,1.178003,112,5,-1,14.0,18.0,1684.0,10168.0,31.0,29.0,1.062137e+04,6.522225e+04,3.0,5.0,90.588921,69.264294,9560.851508,8369.310630,255.0,1.274273e+09,3.424933e+09,255.0,0.000633,0.000506,0.000127,120.0,565.0,1.0,3924.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0,1.0,0.0,Normal
109370,0.350793,-1,5,-1,10.0,8.0,818.0,1144.0,62.0,252.0,1.680763e+04,2.282828e+04,2.0,2.0,38.977000,41.743000,1910.383041,61.134453,255.0,2.655125e+09,1.760676e+09,255.0,0.104911,0.035095,0.069816,82.0,143.0,1.0,150.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,Exploits


In [38]:
y_train_data_32['pred'] = preds

In [39]:
y_train_data_32[y_train_data_32['pred'] == y_train_data_32['label']].shape[0] / y_train_data_32.shape[0]

0.8637962054301603

In [46]:
#feature 22, 19 -> convert to binary (0, 255) và null
#feature 10 -> (0, 252, 29, 60) và null
#feature 9
raw_train["feature9"].value_counts()

feature9
254.0    60740
31.0     20215
62.0     10113
0.0       1540
64.0        72
1.0         31
29.0        14
63.0        12
60.0        12
255.0        8
32.0         4
30.0         1
Name: count, dtype: int64

In [47]:
raw_test["feature9"].value_counts()

feature9
254.0    73316
31.0     23998
62.0     11977
0.0       1815
64.0        90
63.0        36
29.0        18
1.0         18
255.0       18
32.0        17
60.0         9
Name: count, dtype: int64

In [95]:
def data_processing_phase2_prob2(data, phase="train"):
    drop_corr = ['feature5', 'feature13', 'feature6', 'feature14', 'feature22', 'feature30', 'feature35', 'feature32', 'feature36', 'feature37']
    convert_cate_cols = ['feature9', 'feature10', 'feature31']
    data = data.drop(columns = drop_corr)
    for col in convert_cate_cols:
        data[col] = data[col].astype(int).astype(str)
    cate_cols = ['feature2', 'feature3', 'feature4']
    for col in cate_cols:
        dict_count = data[col].value_counts().to_dict()
        list_below = [key for key, value in dict_count.items() if value / data.shape[0] < 0.0005]
        if phase == 'test':
            list_below.append('null')
        data[col] = data[col].replace({value: 'other' for value in list_below})
    return data

In [106]:
[each for each in raw_test_['feature4'].unique() if each not in each for each in raw_train_['feature4'].unique()]

[]

In [107]:
def get_num(n):
    return n

dict_ = {}
dict_[1] = get_num

In [109]:
dict_[1](10)

10

In [1]:
import pickle
with open("../../data/train_data/phase-3/prob-2/category_index.pickle", "rb") as file:
    category_features = pickle.load(file)

In [2]:
with open("../../data/train_data/phase-2/prob-2/category_index.pickle", "wb") as file:
    pickle.dump(category_features, file)

PermissionError: [Errno 13] Permission denied: '../../data/train_data/phase-2/prob-2/category_index.pickle'

In [ ]:
wi